# Part 4: Control Flow
# 11. 반복할 수 있는 객체, 반복자, 제너레이터

## Index
- 14.1 Sentence 버전 #1: 단어 시퀀스
- 14.2 반복형과 반복자
- 14.3 Sentence 버전 #2: 고전적인 반복자
- 14.4 Sentence 버전 #3: 제너레이터 함수
- 14.5 Sentence 버전 #4: 느긋한 구현
- 14.6 Sentence 버전 #5: 제너레이터 표현식
- 14.7 제너레이터 표현식 : 언제 사용하나?
- 14.8 또 다른 예제 : 등차수열 제너레이터

## 14.1 Sentence 버전 #1: 단어 시퀀스
`Sentence` 객체는 입력 받은 텍스트를 단어 단위로 반복하는 객체. 시작은 `Sequence`를 이용하여! `Sequence`는 iterable하니까!

In [1]:
#Example 14-1
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [2]:
#Example 14-2
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

왜 `Sequence`는 iterable일까? 인터프리터가 객체 x를 반복하면 `iter(x)`를 자동으로 호출한다. `iter` 내장 함수는:  
1. 객체가 `__iter__`를 구현했는지 확인하고 반복자를 얻기 위해 호출한다.
2. `__iter__`가 구현 되어있지 않고, `__getitem__`이 구현되어 있으면, 인덱스 0부터 순차적으로 아이템을 가져오는 방식으로 반복자를 만들어낸다.
3. 만약 실패하면, 파이썬은 `TypeError`를 발생하고 대체로 "C object is not iterable,"이라고 말한다.

이게 `Sequence`가 iterable인 이유. `__getitem__`을 가지고 있고 표준 시퀀스는 `__iter__`까지 구현되어 있음. 이것은 duck typing의 극한의 형태. 이걸 goose typing의 관점에서 보면 심플하지만 유연하지 않음. 단순히 `__iter__`이 구현되어 있으면 `iterable`이라고 하면 되기 때문. 서브클래싱이나 등록은 필요하지 않음. `abc.Iterable`에 `__subclasshook__`이 이미 구현되어 있기 때문.

In [5]:
class Foo:
    def __iter__(self):
        pass
    
from collections import abc
issubclass(Foo, abc.Iterable)

True

In [6]:
f = Foo()
isinstance(f, abc.Iterable)

True

보면 정상적으로 서브클래스, 인스턴스로 인식하는 것을 볼 수 있다. 하지만 #14-1은?

In [7]:
issubclass(Sentence, abc.Iterable)

False

### 응 안돼

객체가 iterable인지 명시적으로 확인하고 바로 객체를 iterate하면 의미가 없다. 왜냐하면 어짜피 `TypeError`가 발생하기 때문! 명시적으로 체크하기보다 `TypeError`를 `try/except`로 처리하는게 나을 수도 있다. 나중에 iteration을 하고 싶은 거면 미리 오류를 잡는 거기 때문에 명시적으로 확인하는게 의미가 있다.

## 14.2 반복형과 반복자
파이썬은 iterable에서 iterator를 얻는다.

In [8]:
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it)) #next available item
    except StopIteration: # signal that the iterator is exhausted
        del it
        break

A
B
C


![fig14-1](../images/fig_14-1.png)

In [9]:
#Example 14-3
from abc import *

class Iterator(abc.Iterable):
    
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return true
        return NotImplemented

In [10]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
next(it)

'Pig'

In [11]:
next(it)

'and'

In [12]:
next(it)

'Pepper'

In [13]:
next(it)

StopIteration: 

`iterator`는 `__next__`와 `__iter__`만 요구하기 때문에 `next()`를 돌려보면서 `StopIteration`이 나올 때까지 것밖에 아이템이 남았는지 확인할 수 있는 방법이 없다. 그리고 빠꾸도 없기 때문에 다시 시작하려면 `iter()`에 `iterable`을 넣어서 `iterator`를 만들어야 함.

## 14.3 Sentence 버전 #2: 고전적인 반복자

In [ ]:
#Example 14-4
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return SentenIterator(self.words)
    

class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [ ]:
issubclass(SentenceIterator, abc.Iterator)

정리를 해보자!  
`iterable` : 매번 새로운 `iterator`를 객체화하는 `__iter__` 함수를 가짐
`iterator` : 각각의 아이템을 반환하기 위한 `__next__`, 자기 자신을 반환하는 `__iter__` 메소드를 가짐.  

<blockquote>iterator는 iterable이지만 iterable은 iterator가 아니다.</blockquote>

`Sentence`에 `__next__`를 구현해보면 어떨까? 각각의 `Sentence` 인스턴스가 동시에 `iterable`이고 자신을 순환하는 `iterator`가 될까? 안된다!  
다중 순회를 지원하기 위해서는 `iter(my_iterable)`이 호출될 때 새롭고 독립적인 `iterator`를 만들어내야 한다. 

## 14.4 Sentence 버전 #3: 제너레이터 함수
`SentenceIterator`를 제너레이터로 대체해보자. 

In [ ]:
#Example 14-5
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

In [ ]:
issubclass(Sentence, abc.Iterable)

### Generator는 어떻게 작동할까?
일단 `yield`를 사용하면 generator이다. 호출되면 generator 객체를 반환한다. 

In [ ]:
def gen_123():
    yield 1
    yield 2
    yield 3
gen_123

In [ ]:
gen_123()

In [ ]:
for i in gen_123():
    print(i)

In [ ]:
g = gen_123()
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

In [ ]:
next(g)

A generator function builds a generator object that wraps the body of the function. When we invoke next(…) on the generator object, execution advances to the next yield in the function body, and the next(…) call evaluates to the value yielded when the function body is suspended. Finally, when the function body returns, the enclosing generator object raises StopIteration, in accordance with the Iterator protocol.

In [ ]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
g = iter(gen_AB())

In [ ]:
test = []
for i in g:
    test.append(i)

In [ ]:
test

## 14.5 Sentence 버전 #4: 느긋한 구현
lazy의 좋은 점: 메모리를 아낄 수 있다. 고작 몇 개 iterate 하자고 전체 사이즈의 list를 만들어내고 시작하는 것은 낭비다. `re.findall` -> `re.finditer`로 바꾸면 메모리를 아낄 수 있다!

In [ ]:
#Example 14-7
import re
import reprlib

RE_WORD = re.compile('\w+')


class Sentence:
    
    def __init__(self, next):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 14.6 Sentence 버전 #5: 제너레이터 표현식
제너레이터 표현식을 사용하면 더 짧게 표현할 수 있다. 제너레이터 표현식은 list comprehension의 lazy version이다. 제너레이터는 on demand로 lazy하게 아이템을 생성한다.

In [ ]:
#Example 14-8
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
res1 = [x*3 for x in gen_AB()]

In [ ]:
for i in res1:
    print('-->', i)

In [ ]:
res2 = (x*3 for x in gen_AB())
#소비된 결과가 아니라 generator를 return
res2

In [ ]:
for i in res2: #for 
    print('-->', i)

In [ ]:
#Example 14-9
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

달라진 것은 `__iter__()` 뿐이다! `yield`는 없어졌지만 결과는 동일!

## 14.7 제너레이터 표현식 : 언제 사용하나?
일단은 앞에서 봤던 예들을 들 수가 있다. `Vector` class 선언할 때 사용했었다. `__eq__`, `__hash__`, `__abs__`,.. 이런 것들. 함수를 따로 선언하지 않고, 구문적으로 짧게 쓸 때도 사용함.(#14.9) 하지만 제너레이터 함수가 더 유연함. 당연한 얘기! 표현식 안에는 많이 넣으면 일단 해석이 어려워지고 지저분해진다. shortcut을 사용하려고 사용한 표현식이 shortcut이 아닐 수도 있다..! 표현식은 2X 레버리지 같은 존재. 심플한 건 더 심플하게, 복잡한 건 더 복잡하게 만들어버린다고 생각하면 좋을듯! 필자는 한 줄을 넘어가면 함수로 구현한다고 한다.

## 14.8 또 다른 예제 : 등차수열 제너레이터

In [ ]:
#Example 14-11
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        #step의 type으로 강제된 self.begin의 값을 result에 저장
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index
            #소수점을 누적하면서 생기는 문제를 막기 위해 매번 새로 계산해줌.

In [ ]:
#Example 14-10
ap = ArithmeticProgression(0, 1, 3)
list(ap)

In [ ]:
ap

In [ ]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

In [ ]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

In [ ]:
from fractions import Fraction
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

In [ ]:
from decimal import Decimal
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

In [ ]:
#Example 14-12
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

자, 이제 잘 만들어 놓은 거(itertools)를 갖다 써보자.
### Arithmetic Progression with itertools

In [ ]:
import itertools
gen = itertools.count(1, .5)

In [ ]:
next(gen), next(gen), next(gen)

하지만 이건 끝이 없기 때문에
```Python
for i in gen:
    print(i)
```
같은 걸 한다던가,
`list(gen)` 같은 것을 해버리면, 가용 메모리보다 큰 작업을 하기 때문에 실패한다. for문은 계속 돌겠지만..?  
그러면 이제 `itertools.takewhile`을 써보자. 다른 generator를 소비해서 조건이 `False`가 될 때까지 돌려서 generator를 생성한다.

In [ ]:
gen = itertools.takewhile(lambda n: n<3, itertools.count(1, .5))
list(gen)

이걸 사용해서 Example 14-12에 적용해보자.

In [ ]:
#Example 14-13
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

또, `yield`문은 사라졌지만 여전히 generator를 return한다. 결론은, 원리를 알고 있는 걸 잘 갖다 쓰자는 말!